In [1]:
import tensorflow as tf
import keras
import numpy as np
import time
import random
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Flatten, LSTM, GRU, SimpleRNN, Conv2D, MaxPooling2D, Flatten, Dropout, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

from collections import deque


from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error as mse
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output

import qkeras
from qkeras import *

import hls4ml
import nnlar
from nnlar.datashaper import DataShaper

2022-06-01 16:55:55.573691: I tensorflow/core/util/util.cc:168] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-06-01 16:55:55.579390: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-01 16:55:55.579406: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


name convert optimizers ['fuse_bias_add', 'remove_useless_transpose', 'output_rounding_saturation_mode', 'qkeras_factorize_alpha', 'extract_ternary_threshold', 'fuse_consecutive_batch_normalization'] File: flow.py Line: 23
name optimize optimizers ['eliminate_linear_activation', 'fuse_consecutive_batch_normalization', 'fuse_batch_normalization', 'replace_multidimensional_dense_with_conv'] File: flow.py Line: 23
vivado:merge_batch_norm_quantized_tanh Get_Optimizer, optimizer/optimizer.py ligne: 168
vivado:quantize_dense_output Get_Optimizer, optimizer/optimizer.py ligne: 168
vivado:batchnormalizationquantizedtanh_config_template Get_Optimizer, optimizer/optimizer.py ligne: 168
vivado:batchnormalizationquantizedtanh_function_template Get_Optimizer, optimizer/optimizer.py ligne: 168
vivado:clone_output Get_Optimizer, optimizer/optimizer.py ligne: 168
vivado:clone_function_template Get_Optimizer, optimizer/optimizer.py ligne: 168
vivado:optimize_pointwise_conv Get_Optimizer, optimizer/opti

/atlas/bonnet/hls4ml/hls4ml/hls4ml/hls4ml/converters/__init__.py:15: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


In [15]:
import importlib

def path_to_datashaper(steps): 
    module = importlib.import_module(f"time_step_test.datashapers.datashaper_{steps}")
    
    return module

In [19]:
ds = path_to_datashaper(30).DataShaper.from_h5("data/rdgap_mu140.h5")
x, x_val, x_test, y, y_val, y_test3 = ds()
ds = path_to_datashaper(5).DataShaper.from_h5("data/rdgap_mu140.h5")
x, x_val, x_test, y, y_val, y_test = ds()

shapes (1999970, 30, 1) (1999970, 1)
shapes (899956, 30, 1) (99969, 30, 1) (999985, 30, 1)
shapes (1999995, 5, 1) (1999995, 1)
shapes (899992, 5, 1) (99995, 5, 1) (999998, 5, 1)


In [22]:

import importlib
import nnDumper_standalone
importlib.reload(nnDumper_standalone)


m_t = 16
model2 = np.load('time_step_test/tests/predicts/model_timestep=30_v1.npy')

dump_preds = nnDumper_standalone.dumper(start = 1000010, name = "timestep", bt_len=80)

dump_preds.set_data(true = ds.hit, data = ds.dig, sig = ds.sig, ofmax = ds.ofmax)

dump_preds.set_preds('time step 30', m_t*model2.flatten())


dump_preds.runme()

uproot version: 4.2.3
nnDumper::Enter gather_results
Collect general objects:
nnDumper::Exit gather_results
nnDumper::Enter dump_results
nnDumper::Make TTree
nnDumper::Done with tree, skipping THs
nnDumper::Exit dump_results


In [19]:
   
import importlib
import nnDumper_standalone
importlib.reload(nnDumper_standalone)

ds = DataShaper.from_h5("data/rdgap_mu140.h5")
x, x_val, x_test, y, y_val, y_test = ds()
m_t = 16
model1 = np.load('rnn/tests/qpredicts_test/qtrained/qmodels_units=8_epoch=200_patiencES=8_patienceRLR=5_delta=1e-08,<8,0>v1.npy')
model2 = np.load('rnn/tests/qpredicts_test/qtrained/qmodels_units=8_epoch=200_patiencES=12_patienceRLR=5_delta=1e-08,<8,0>v2.npy')
model3 = np.load('rnn/tests/qpredicts_test/qtrained/qmodels_units=8_epoch=200_patiencES=16_patienceRLR=5_delta=1e-08,<8,0>v0.npy')
model4 = np.load('rnn/tests/qpredicts_test/qtrained/qmodels_units=8_epoch=200_patiencES=20_patienceRLR=5_delta=1e-08,<8,0>v0.npy')
model5 = np.load('rnn/qpredicts_epoch=4/qpredicts<8,0>.npy')
dump_preds = nnDumper_standalone.dumper(start = 999997, name = "qpatience_compare", bt_len=80)

dump_preds.set_data(true = ds.hit, data = ds.dig, sig = ds.sig, ofmax = ds.ofmax)

dump_preds.set_preds('rnn_patience=8', m_t*model1.flatten())
dump_preds.set_preds('rnn_patience=12', m_t*model2.flatten())
dump_preds.set_preds('rnn_patience=16', m_t*model3.flatten())
dump_preds.set_preds('rnn_patience=20', m_t*model4.flatten())
dump_preds.set_preds('rnn_ref', m_t*model5.flatten())

dump_preds.runme()

uproot version: 4.2.3
shapes (1999995, 5, 1) (1999995, 1)
shapes (899992, 5, 1) (99995, 5, 1) (999998, 5, 1)
nnDumper::Enter gather_results
Collect general objects:
nnDumper::Exit gather_results
nnDumper::Enter dump_results
nnDumper::Make TTree
nnDumper::Done with tree, skipping THs
nnDumper::Exit dump_results
